In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import keras
from keras.layers import Dropout,BatchNormalization,LSTM,Bidirectional,GlobalMaxPool1D,Input,Activation,Flatten,Embedding,Dense,concatenate,Conv1D,MaxPooling1D
import string
import re
from tqdm import tqdm
import nltk
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords
import spacy
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,f1_score,recall_score,precision_score,classification_report
import os
from keras.models import Model
import seaborn as sns
from matplotlib import pyplot as plt
from wordcloud import WordCloud, STOPWORDS
import warnings
warnings.filterwarnings('ignore')
import tensorboard
from textblob import TextBlob
import os
import tensorboard
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
import datetime
from keras.initializers import he_normal
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from prettytable import PrettyTable

In [2]:
!pip install ktrain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.3/25.3 MB 17.1 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 23.5 MB/s eta 0:00:0000:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 263.7/263.7 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 33.4 MB/s eta 0:00:0000:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.8/468.8 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 24.0 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for ktrain: filename=ktrain-0.31.6-py3-none-any.whl size=25312

In [4]:
data_new=pd.read_csv("../input/data-pre-3/data_preprocessed.csv")
data_new.shape

(1010660, 10)

In [5]:
data_new1=data_new.drop(['Unnamed: 0'],axis=1)
data_new1=data_new1.dropna()

In [6]:
data=data_new1.sample(100000)

In [7]:

y=data['label']
X=data.drop(['label'],axis=1)

In [8]:
X_train,X_test,y_train,y_test=train_test_split(X,y,stratify=y,random_state=42,test_size=0.3,shuffle=True)
print((X_train.shape),y_train.shape)
print((X_test.shape),y_test.shape)

(70000, 8) (70000,)
(30000, 8) (30000,)


In [9]:
class_names=[0,1]

y_train=np.asarray(y_train)
y_test=np.asarray(y_test)

In [10]:
import ktrain
from ktrain import text
model_name = 'distilbert-base-uncased'
t = text.Transformer(model_name, maxlen=40, class_names=class_names)
trn = t.preprocess_train(list(X_train['comment']), y_train)
val = t.preprocess_test(list(X_test['comment']), y_test)
model = t.get_classifier()
learner = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size=256)

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

preprocessing train...
language: en
train sequence lengths:
	mean : 6
	95percentile : 14
	99percentile : 21


Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 6
	95percentile : 14
	99percentile : 21


Downloading:   0%|          | 0.00/363M [00:00<?, ?B/s]

In [11]:
learner.fit_onecycle(2e-5, 3)



begin training using onecycle policy with max lr of 2e-05...
Epoch 1/3
274/274 [==============================] - 233s 721ms/step - loss: 0.6467 - accuracy: 0.6124 - val_loss: 0.6027 - val_accuracy: 0.6707
Epoch 2/3
274/274 [==============================] - 214s 711ms/step - loss: 0.5785 - accuracy: 0.6907 - val_loss: 0.5810 - val_accuracy: 0.6896
Epoch 3/3
274/274 [==============================] - 214s 710ms/step - loss: 0.5305 - accuracy: 0.7320 - val_loss: 0.5799 - val_accuracy: 0.6965


In [12]:
learner.model.save_pretrained('.// model_bert')

In [13]:
learner.validate(class_names=t.get_classes())

              precision    recall  f1-score   support

           0       0.68      0.74      0.71     15023
           1       0.71      0.65      0.68     14977

    accuracy                           0.70     30000
   macro avg       0.70      0.70      0.70     30000
weighted avg       0.70      0.70      0.70     30000



array([[11113,  3910],
       [ 5196,  9781]])

In [14]:
predictor = ktrain.get_predictor(learner.model, preproc=t)

In [15]:
predictor.predict('ayush is awesome')

0

In [16]:
predictor.predict('I work 40 hours a week for me to be this poor.')

1

In [17]:
predictor.predict('ayush khandelwal')

0

In [18]:
predictor.predict('mohit is awesome')

0

In [19]:
predictor.save('.//model_bert_1')

In [20]:
reloaded_predictor = ktrain.load_predictor('.//model_bert_1')

In [23]:
reloaded_predictor.predict('mohit is awesome')

0

In [25]:
reloaded_predictor.predict('Thats just what I needed today!')

1

In [26]:
reloaded_predictor.predict('Well, what a surprise.')

1

In [27]:
reloaded_predictor.predict('Is it time for your medication or mine?')

0

In [28]:
reloaded_predictor.predict('Really, Sherlock? No! You are clever.')

1

In [ ]:
reloaded_predictor.predict('mohit is awesome')

In [29]:
reloaded_predictor.predict('Nice perfume. How long did you marinate in it?')

0

In [30]:
reloaded_predictor.predict('Very good; well done!')

0

In [31]:
reloaded_predictor.predict('I like spaghetti.')

0

In [32]:
reloaded_predictor.predict('He reads many books.')

0

In [33]:
reloaded_predictor.predict('I like spaghetti for dinner.')

0